In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv('../Data_hysso\desk_deskdefault\hysso_desktop_counts.csv',
                 index_col=0)
df

In [ ]:
grupos = (pd.DataFrame(df.columns.to_series()
             .str.split('_').to_dict())).iloc[0,:].to_list()
grupos

In [ ]:
unicos, cuentas = np.unique(grupos, return_counts=True)
criba = np.min(cuentas)
df_criba = df.loc[df.sum(axis=1) > criba, :].T
df_criba

CLR

In [ ]:
from scipy.stats.mstats import gmean
dfclr = df_criba.replace(to_replace=0, value=1e-9).div(df_criba.replace(to_replace=0, value=1e-9).apply(gmean, axis=1), axis=0).apply(np.log)  # tabla transformada
dfclr

In [ ]:
dfclr.sum(axis=1)

L_1_1     1.364242e-12
S_1_2     6.821210e-12
L_1_3     1.364242e-12
M_1_4     6.366463e-12
C_1_5     5.911716e-12
              ...     
S_5_20    9.094947e-13
S_5_21    3.183231e-12
S_5_22    0.000000e+00
S_5_23    1.364242e-12
S_5_24   -9.094947e-13
Length: 72, dtype: float64

Separar por grupos

In [ ]:
control = dfclr.loc[
    [sam for sam in dfclr.index if 'C_' in sam],
    :]
leve = dfclr.loc[
    [sam for sam in dfclr.index if 'L_' in sam],
    :]
moderado = dfclr.loc[
    [sam for sam in dfclr.index if 'M_' in sam],
    :]
severo = dfclr.loc[
    [sam for sam in dfclr.index if 'S_' in sam],
    :]

## Análisis de la varianza (ANOVA)

In [ ]:
from scipy.stats import f_oneway, false_discovery_control, permutation_test

In [ ]:
def fisher(*x, axis=0) :
    fstat, pvalue = f_oneway(*x, axis=axis)
    return fstat

In [ ]:
anova = permutation_test(data = (control, leve, moderado, severo),
                         statistic = fisher,
                         axis = 0,
                         random_state = 42,
                         vectorized = True,
                         alternative = 'greater')

In [ ]:
otu_aov = pd.DataFrame({
    'otu': list(dfclr.columns),
    'F_stat': anova.statistic,
    'p-valor': anova.pvalue,
    'p-valor(BH)': false_discovery_control(anova.pvalue, method='bh')
})
otu_aov

In [ ]:
otu_aov_sig = otu_aov[otu_aov['p-valor(BH)'] < 0.1]
print(otu_aov_sig.shape)
print(otu_aov_sig)